In [189]:
import os
import time

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import pyswarms as ps
import tensorflow as tf
from keras import optimizers
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
from pyswarms.utils.plotters import plot_cost_history
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [190]:
matplotlib.style.use('ggplot')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示中文标签
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['xtick.direction'] = 'in'  # 将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'  # 将y轴的刻度方向设置向内

In [191]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


In [192]:
# load dataset
df = pd.read_excel("202106有機自動加藥數據統計(新).xlsx" , header=0, index_col=0)
df = df[[ '銅在線濃度(mg/L)','OOO','pH','ph槽ORP','NaS_ORP','NaS','FeSO4']]

In [193]:
df

,銅在線濃度(mg/L),OOO,pH,ph槽ORP,NaS_ORP,NaS,FeSO4
時間,,,,,,,
2021-06-01 18:10:00.000,7.070000,8017.94,7.49,7.52,-476,9.595294,96.984951
2021-06-01 18:38:32.781,7.093792,7769.10,7.49,7.52,-476,9.609958,97.096244
2021-06-01 18:38:36.790,7.093847,7769.10,7.48,7.52,-476,9.620155,97.090278
2021-06-01 18:59:10.120,7.110972,7832.75,7.48,7.52,-476,9.630800,97.170735
2021-06-01 18:59:12.117,7.111000,7838.54,7.48,7.52,-476,9.630817,97.170866
...,...,...,...,...,...,...,...
2021-06-30 19:50:20.596,9.759152,7624.42,7.96,5.21,-156,10.958985,117.887106
2021-06-30 20:23:47.670,9.638594,7960.07,7.96,5.21,-156,10.857909,116.696064
2021-06-30 20:25:01.661,9.634149,7960.07,7.96,5.21,-24,10.854213,116.652627


In [194]:
Y = df.iloc[:, 0]

In [195]:
Y

時間
2021-06-01 18:10:00.000    7.070000
2021-06-01 18:38:32.781    7.093792
2021-06-01 18:38:36.790    7.093847
2021-06-01 18:59:10.120    7.110972
2021-06-01 18:59:12.117    7.111000
                             ...   
2021-06-30 19:50:20.596    9.759152
2021-06-30 20:23:47.670    9.638594
2021-06-30 20:25:01.661    9.634149
2021-06-30 21:30:00.000    9.400000
2021-06-30 23:00:00.000    9.800000
Name: 銅在線濃度(mg/L), Length: 1370, dtype: float64

In [196]:
X =  df[['OOO','pH','ph槽ORP','NaS_ORP','NaS','FeSO4']]

In [197]:
X

,OOO,pH,ph槽ORP,NaS_ORP,NaS,FeSO4
時間,,,,,,
2021-06-01 18:10:00.000,8017.94,7.49,7.52,-476,9.595294,96.984951
2021-06-01 18:38:32.781,7769.10,7.49,7.52,-476,9.609958,97.096244
2021-06-01 18:38:36.790,7769.10,7.48,7.52,-476,9.620155,97.090278
2021-06-01 18:59:10.120,7832.75,7.48,7.52,-476,9.630800,97.170735
2021-06-01 18:59:12.117,7838.54,7.48,7.52,-476,9.630817,97.170866
...,...,...,...,...,...,...
2021-06-30 19:50:20.596,7624.42,7.96,5.21,-156,10.958985,117.887106
2021-06-30 20:23:47.670,7960.07,7.96,5.21,-156,10.857909,116.696064
2021-06-30 20:25:01.661,7960.07,7.96,5.21,-24,10.854213,116.652627


In [198]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25, random_state=420)
# print(Xtrain.describe().T)

ss_x = StandardScaler()
Xtrain = ss_x.fit_transform(Xtrain)
Xtest = ss_x.transform(Xtest)

X = ss_x.transform(X)
ss_y = StandardScaler()
Ytrain = ss_y.fit_transform(Ytrain.values.reshape(-1, 1))
Y = ss_y.fit_transform(Y.values.reshape(-1, 1))
Ytest = ss_y.transform(Ytest.values.reshape(-1, 1))


Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], 1))
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtrain.shape[1], 1))

Ytrain = np.reshape(Ytrain, (Ytrain.shape[0], ))
Ytest = np.reshape(Ytest, (Ytest.shape[0], ))



In [201]:
timesteps = 6
features = 1
print(Xtrain.shape,Ytrain.shape)

# Write Keras model LSTM layers and compile the model using SGD
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps, features)))
model.add(Dense(1))

(1027, 6, 1) (1027,)


In [202]:
from tensorflow.keras import optimizers

In [203]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

C:\Users\2520\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [206]:
model.summary()
total_param = 68737
# Use model fit command to train the model
t0 = time.time()
result_sgd = model.fit(Xtrain, Ytrain, batch_size=4, epochs=50, verbose=0)
t1 = time.time()

print("***************************")
print()
print("LSTM Model")
print("Time taken to train the model: ", t1 - t0, "secs.")
print("Error:", result_sgd.history['loss'][-1])


Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_23 (LSTM)              (None, 128)               66560     
                                                                 
 dense_21 (Dense)            (None, 1)                 129       
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
***************************

LSTM Model
Time taken to train the model:  37.2939555644989 secs.
Error: 0.0005899481475353241


In [207]:

model_weights = np.array([model.layers[0].get_weights(), model.layers[1].get_weights()])
shape = np.array(
    [model_weights[0][0].shape, model_weights[0][1].shape, model_weights[0][2].shape, model_weights[1][0].shape,
     model_weights[1][1].shape])


def func(vector_x):
    init_index = 0
    end_index = 0
    end_index += shape[0][0] * shape[0][1]
    model_weights[0][0] = vector_x[init_index:end_index].reshape(shape[0])
    init_index = end_index
    end_index += shape[1][0] * shape[1][1]
    model_weights[0][1] = vector_x[init_index:end_index].reshape(shape[1])
    init_index = end_index
    end_index += shape[2][0]
    model_weights[0][2] = vector_x[init_index:end_index]
    init_index = end_index
    end_index += shape[3][0] * shape[3][1]
    model_weights[1][0] = vector_x[init_index:end_index].reshape(shape[3])
    init_index = end_index
    end_index += shape[4][0]
    model_weights[1][1] = vector_x[init_index:end_index]
    model.layers[0].set_weights(model_weights[0])
    model.layers[1].set_weights(model_weights[1])
    pso_predict = model.predict(X)
    error = mean_squared_error(Y, pso_predict)
    return error

C:\Users\2520\AppData\Local\Temp/ipykernel_15036/3367695925.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  model_weights = np.array([model.layers[0].get_weights(), model.layers[1].get_weights()])
C:\Users\2520\AppData\Local\Temp/ipykernel_15036/3367695925.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  shape = np.array(


In [208]:
def swarm_func(x):
    n_particles = x.shape[0]
    j = [func(x[i]) for i in range(n_particles)]
    return np.array(j)


# initialization
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=80, dimensions=total_param,
                                    options=options)
t2 = time.time()
result_pso = optimizer.optimize(model, iters=120)
t3 = time.time()

print("Partical Swarm Optimization")
print("Time taken to train the model: ", t3 - t2, "secs.")
print("Error:", result_pso[0])
print()
print("***************************")

print()
plot_cost_history(optimizer.cost_history)

2021-11-30 16:00:06,837 - pyswarms.single.global_best - INFO - Optimize for 120 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/120


ValueError: Input 0 of layer "sequential_23" is incompatible with the layer: expected shape=(None, 6, 1), found shape=(80, 68737)

In [45]:
plt.show()

In [46]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps, features)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [47]:
model